In [73]:
import numpy as np

In [74]:
with open("log.txt") as f:
    data = []
    lines = f.readlines()
    for line in lines[:-1]:
        data.append(np.array(line[1:-2].split(" ")).astype(int))

In [75]:
data = np.array(data)

In [76]:
data.shape

(69017, 25)

In [77]:
data = np.unique(data, axis=0)

In [78]:
data.shape

(63375, 25)

In [79]:
from minimax_agent.evaluation import evaluation

In [81]:
score = []
for i, row in enumerate(data):
    if i % 1000 == 0:
        print(i)
    score.append(evaluation(np.reshape(row, (5,5)), 5))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000


In [83]:
score = np.array(score)

In [85]:
# np.save('score.npy', score)

In [86]:
import collections

collections.Counter(score)

Counter({1.0: 16216,
         0.0: 23016,
         -1.0: 11392,
         2.0: 5196,
         -2.0: 3570,
         -3.0: 719,
         3.0: 1050,
         4.0: 149,
         -inf: 900,
         inf: 1050,
         5.0: 13,
         -4.0: 91,
         -6.0: 1,
         -5.0: 9,
         6.0: 3})

In [87]:
score.shape

(63375,)

In [88]:
X = data
y = score

In [89]:
y = np.where(score==np.inf, 0, y)
y = np.where(score==-np.inf, 0, y)

In [90]:
collections.Counter(y)

Counter({1.0: 16216,
         0.0: 24966,
         -1.0: 11392,
         2.0: 5196,
         -2.0: 3570,
         -3.0: 719,
         3.0: 1050,
         4.0: 149,
         5.0: 13,
         -4.0: 91,
         -6.0: 1,
         -5.0: 9,
         6.0: 3})

In [96]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape)
print(y_train.shape)

(50700, 25)
(50700,)


In [92]:
from tensorflow import keras

In [139]:
model = keras.Sequential(
    [
        keras.Input(shape=(25,)),
        keras.layers.Dense(64, kernel_initializer='random_normal', activation="relu"),
        keras.layers.Dense(64, kernel_initializer='random_normal', activation="relu"),
        keras.layers.Dense(64, kernel_initializer='random_normal', activation="relu"),
        keras.layers.Dense(32, kernel_initializer='random_normal', activation="relu"),
        keras.layers.Dense(16, kernel_initializer='random_normal', activation="relu"),
        keras.layers.Dense(1)
    ]
)

model.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer=keras.optimizers.Adam(learning_rate=0.01),
    metrics=["accuracy"],
)

In [140]:
history = model.fit(X_train, y_train, batch_size=64, epochs=20, validation_split=0.2)

Epoch 1/20
634/634 [==============================] - 1s 2ms/step - loss: 1.2989 - accuracy: 0.3934 - val_loss: 1.2694 - val_accuracy: 0.3905
Epoch 2/20
634/634 [==============================] - 1s 2ms/step - loss: 1.2984 - accuracy: 0.3933 - val_loss: 1.2685 - val_accuracy: 0.3905
Epoch 3/20
634/634 [==============================] - 1s 2ms/step - loss: 1.2983 - accuracy: 0.3933 - val_loss: 1.2680 - val_accuracy: 0.3905
Epoch 4/20
634/634 [==============================] - 1s 2ms/step - loss: 1.2984 - accuracy: 0.3933 - val_loss: 1.2682 - val_accuracy: 0.3905
Epoch 5/20
634/634 [==============================] - 1s 2ms/step - loss: 1.2984 - accuracy: 0.3933 - val_loss: 1.2681 - val_accuracy: 0.3905
Epoch 6/20
634/634 [==============================] - 1s 2ms/step - loss: 1.2988 - accuracy: 0.3933 - val_loss: 1.2682 - val_accuracy: 0.3905
Epoch 7/20
634/634 [==============================] - 1s 2ms/step - loss: 1.2985 - accuracy: 0.3933 - val_loss: 1.2680 - val_accuracy: 0.3905
Epoch 

In [141]:
test_scores = model.evaluate(X_test, y_test, verbose=2)

397/397 - 0s - loss: 1.2832 - accuracy: 0.3988


In [134]:
test_input = np.array("0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 2 2 0 2 1 0 2 2 1 0".split(" ")).astype(int)
test_input

array([0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 1, 0, 2,
       2, 1, 0])

In [135]:
test_input = np.expand_dims(test_input, axis=0)
model(test_input)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.22255683]], dtype=float32)>

In [136]:
evaluation(test_input.reshape(5,5), 5)

2